In [ ]:
import json

from tqdm import tqdm


def search_dfs(subgraph, topic_entities, answers, path):
    # Initialize a stack, each element in the stack is a list, the list contains a topic entity and a number 0, the number 0 represents the current hop
    head_stack = [[topic_entity, 0] for topic_entity in topic_entities]
    hop = len(path)

    while head_stack:
        current_head = head_stack.pop()
        # If the popped entity is in the answer, return True
        if current_head[0] in answers:
            return True
        current_hop = current_head[1]
        if current_hop >= hop:
            continue
        current_rel = path[current_hop]
        if current_head[0] not in subgraph:
            continue
        dict = subgraph[current_head[0]]

        # for t in tuples:
        #     if t[0] == current_rel:
        #         tails = t[1]
        #         for tail in tails:
        #             head_stack.append([tail, current_hop+1])
        #         break
        for key, value in dict.items():
            if key == current_rel:
                tails = value
                for tail in tails:
                    head_stack.append([tail, current_hop + 1])
                break
    return False


input_subgraph = "data/webqsp/subgraph/subgraph.json"
input_relation_path = "data/webqsp/tmp/step3.json"

subgraph = {}
with open(input_subgraph, "r") as f:
    for line in f.readlines():
        subgraph.update(json.loads(line))

paths = []
with open(input_relation_path, "r") as f:
    for line in f.readlines():
        paths.append(json.loads(line))

total = len(paths)
partial_hit = 0
hit = 0

for qa_path_score in tqdm(paths):
    path_and_score_list = qa_path_score["path_and_score_list"]
    topic_entities = qa_path_score["topic_entities"]
    answers = qa_path_score["answers"]
    flag = 0

    path_list = [
        path_and_score["path"]
        for path_and_score in path_and_score_list
        if path_and_score["score"] > 0.01
    ]

    if not path_list:
        total -= 1
        continue

    for path in path_list:
        if search_dfs(subgraph, topic_entities, answers, path):
            flag += 1

    if flag > 0:
        partial_hit += 1
        if flag == len(path_list):
            hit += 1

# hit: all paths hit an answer, partial_hit: at least one path hit an answer
print(f"hit: {hit}, partial_hit: {partial_hit}, total: {total}")